# NLP TO SQL

In this project, we’ll build out a simple proof of concept that would allow a non-technical person to directly interact with a database and retrieve information without knowing SQL.


#### Import Libraries

In [22]:
import os
import openai
import pandas as pd
from psutil import sensors_temperatures
from sqlalchemy import create_engine
from sqlalchemy import text

#### Get OpenAI API key

In [13]:
openai.api_key = os.getenv('OPENAI_API_KEY')

#### Tabular Data Setup

In [17]:
df = pd.read_csv("data/sales_data_sample.csv")
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME
0,10107,30,95.70,2871.00,2/24/2003 0:00,1,2,2003,Motorcycles,2125557818,897 Long Airport Avenue,NYC,NY,10022,USA,Yu,Kwai
1,10121,34,81.35,2765.90,5/7/2003 0:00,2,5,2003,Motorcycles,26.47.1555,59 rue de l'Abbaye,Reims,NaN,51100,France,Henriot,Paul
2,10134,41,94.74,3884.34,7/1/2003 0:00,3,7,2003,Motorcycles,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,NaN,75508,France,Da Cunha,Daniel
3,10145,45,83.26,3746.70,8/25/2003 0:00,3,8,2003,Motorcycles,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,USA,Young,Julie
4,10159,49,100.00,5205.27,10/10/2003 0:00,4,10,2003,Motorcycles,6505551386,7734 Strong St.,San Francisco,CA,NaN,USA,Brown,Julie


In [19]:
df.groupby("QTR_ID").sum()['SALES']

C:\Users\Elif\AppData\Local\Temp\ipykernel_10864\2501432.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby("QTR_ID").sum()['SALES']


QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64

In [23]:
# Temp DB in RAM
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [24]:
# Push pandas df -> Temp DB
data = df.to_sql(name='Sales', con=temp_db)

2023-03-25 18:52:41,754 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-25 18:52:41,756 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-03-25 18:52:41,757 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-25 18:52:41,759 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-03-25 18:52:41,760 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-25 18:52:41,762 INFO sqlalchemy.engine.Engine ROLLBACK
2023-03-25 18:52:41,765 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-25 18:52:41,767 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT
)


2023-03-25 18:52:41,768 INFO sqlalche

In [27]:
qry1 = 'SELECT * FROM Sales'
qry2 = 'SELECT SUM(SALES) FROM Sales'

In [30]:
# SQL query on Temp DB
with temp_db.connect() as conn:
    # make the connection
    # run code indentation/block
    result = conn.execute(text(qry2))
    # auto close connection

2023-03-25 18:59:11,463 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-25 18:59:11,464 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales
2023-03-25 18:59:11,465 INFO sqlalchemy.engine.Engine [generated in 0.00262s] ()
2023-03-25 18:59:11,467 INFO sqlalchemy.engine.Engine ROLLBACK


In [31]:
result.all()

[(10032628.85000001,)]

#### Natural Language Request

[Best practices](https://platform.openai.com/docs/guides/code/best-practices)
[Examples - SQL Translate](https://platform.openai.com/examples/default-sql-translate)

\### Postgres SQL tables, with their properties:
\#
\# Employee(id, name, department_id)
\# Department(id, name, address)
\# Salary_Payments(id, employee_id, amount, date)
\#
\### A query to list the names of the departments which employed more than 10 employees in the last 3 months
SELECT

In [36]:
# Used #s, to learn why look best practices and examples - SQL translate

def create_table_definition(df):
    prompt = f"""### sqlite SQL tables, with its properties:
    #
    # Sales({",".join(str(col) for col in df.columns)})
    #
    """

    return prompt

In [37]:
",".join(str(col) for col in df.columns)

'ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME'

In [38]:
print(create_table_definition(df))

### sqlite SQL tables, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #
    


In [39]:
def prompt_input():
    nlp_text = input('Enter the info you want:')
    return nlp_text

In [40]:
prompt_input()

'grab all sales per quarter'

In [44]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition(df)
    query_init_string = f'### A query to answer: {query_prompt}\nSELECT'
    return  definition+query_init_string

In [48]:
nlp_text = prompt_input()
combine_prompts(df, nlp_text)

### sqlite SQL tables, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #
    ### A query to answer: return the sum of SALES per POSTALCODE
SELECT


#### Text Completion API - Parameters Overview

[Details of every model](https://platform.openai.com/docs/model-index-for-researchers)
[Models](https://platform.openai.com/docs/models)

**OpenAI Completion Call Parameters**

**Model**
When trying to decide on a model, consider the following:
- Your cost and how often you will call the model.
- How many tokens you will pass to the model.
- The importance of accuracy in the results received from the model.

**Prompt**
- More than anything, this is by far the most important parameter you will use, as it basically dictates what the model will return as it attempts to complete your text.
- This is also the hardest to get correct, as it usually takes some experimentation.

Advice:
- Take advantage of the “playground” OpenAI provides to understand how changing the prompt can affect output.
- Always do a Google search to see if anyone has “discovered” good initial prompts based on what you are trying to build.


**Temperature**
- Higher values means the model will take more risks.
- Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.
- In the case of simpler tasks with a clear correct answer, you will want to lean more towards 0.

**Max Tokens**
- The maximum number of tokens to generate in the completion.
- The token count of your prompt plus max_tokens cannot exceed the model's context length.
- Most models have a context length of 2048 tokens (except for the newest models, which support 4096).

**Top P**
- An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
- Alter Top P or Temperature but <strong>not both</strong>.

**N**
- How many completions to generate for each prompt.
- Keep in mind that this is essentially the same thing as running the same prompt multiple times, so this could consume your token quota quickly, which costs money!
- We will pretty much always set this to 1, indicating we only want 1 completion.

**Frequency Penalty**
- Number between -2.0 and 2.0.
- Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.

**Presence Penalty**
- Number between -2.0 and 2.0.
- Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.

**Presence and Frequency Penalties**
- The presence penalty is a one-off additive contribution that applies to all tokens that have been sampled at least once and the frequency penalty is a contribution that is proportional to how often a particular token has already been sampled.
- Reasonable values for the penalty coefficients are around 0.1 to 1 if the aim is to just reduce repetitive samples somewhat.
- If the aim is to strongly suppress repetition, then one can increase the coefficients up to 2, but this can noticeably degrade the quality of samples.
- Negative values can be used to increase the likelihood of repetition.
- Depending on your use case, you may need to play around with these values to achieve the right balance.
- Full information on the formula these parameters are a part of: [Link](https://platform.openai.com/docs/api-reference/parameter-details)

#### Open AI Call and Request Handling

In [63]:
openai.api_key_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'openai-api-key.txt'))

In [65]:
response = openai.Completion.create(
    model='text-davinci-003',
    prompt=combine_prompts(df, nlp_text),
    temperature=0,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0,
    stop=['#',';']
)

In [72]:
print(response['choices'][0]['text'])

 POSTALCODE, SUM(SALES)
FROM Sales
GROUP BY POSTALCODE


In [73]:
def handle_response(response):
    query = response['choices'][0]['text']
    return "SELECT"+query if query.startswith(" ") else query

In [75]:
print(handle_response(response))

SELECT POSTALCODE, SUM(SALES)
FROM Sales
GROUP BY POSTALCODE


In [76]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-03-25 20:44:27,081 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-25 20:44:27,083 INFO sqlalchemy.engine.Engine SELECT POSTALCODE, SUM(SALES)
FROM Sales
GROUP BY POSTALCODE
2023-03-25 20:44:27,085 INFO sqlalchemy.engine.Engine [generated in 0.00388s] ()
2023-03-25 20:44:27,089 INFO sqlalchemy.engine.Engine ROLLBACK


In [77]:
result.all()

[(None, 272407.14),
 ('10022', 560787.7699999998),
 ('10100', 94117.26000000002),
 ('106-0032', 120562.73999999996),
 ('1203', 117713.55999999998),
 ('1227 MM', 94015.73),
 ('13008', 74936.14),
 ('1734', 145041.6),
 ('2', 57756.43),
 ('2060', 153996.13000000003),
 ('2067', 151570.98000000004),
 ('21240', 111250.37999999996),
 ('24067', 85555.98999999998),
 ('24100', 137955.72000000003),
 ('28023', 170257.33000000005),
 ('28034', 912294.1100000002),
 ('3004', 200995.40999999997),
 ('31000', 70488.44),
 ('3150', 64591.46000000001),
 ('4101', 59469.11999999999),
 ('4110', 116599.19),
 ('41101', 54723.62),
 ('42100', 142601.33000000002),
 ('44000', 204304.86),
 ('5020', 149798.63),
 ('50553', 207874.86),
 ('50739', 100306.58),
 ('51003', 154069.65999999997),
 ('51100', 135042.94),
 ('51247', 139243.99999999994),
 ('530-0003', 67605.07),
 ('58339', 165255.20000000004),
 ('59000', 69052.41),
 ('60528', 85171.58999999998),
 ('62005', 131685.30000000002),
 ('67000', 80438.48),
 ('69004', 14287